# 必要な準備を行う

実験中のワークフロー実行のために必要な準備を行います。  
上から順番に実行してください。

## 1. ユーザー認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行行し、画面の表示に沿ってデータガバナンス機能に登録したユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
%cd ~/EX-WORKFLOW/util/scripts
import json
from scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    email = input("メールアドレス：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

!git config --global user.name $name
!git config --global user.email $email

## 2. データ同期のための設定をする

この手順では、今の実行環境とデータガバナンス機能のリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json
from scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('Public key is ready.')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('Public key is ready before time.')

In [ ]:
# sibling url をsshに変更する
import os
import tempfile
from datalad import api
from IPython.display import clear_output

remote_info = 'remote.txt'

with tempfile.TemporaryDirectory() as tmpdir:
    # 使い終わったらPython側で消してくれるTemporaryDirectoryを利用
    # refs: https://qiita.com/everylittle/items/aa7c6f612ff0a9db7f01
    tmppath = os.path.join(tmpdir, remote_info)
    !git remote show origin > $tmppath
    with open(tmppath, 'r') as f:
        lines = [s.strip() for s in f.readlines()]
        
        # FIXME: GitHub等リポジトリにも対応するためハードコーディングを解消
        ssh_push_url = lines[2].replace('Push  URL: http://dg02.dg.rcos.nii.ac.jp/', 'ssh://root@dg02.dg.rcos.nii.ac.jp:3001/') 

api.siblings(action='add', name='gin', url=ssh_push_url)
clear_output()
print('SSH connection is ready.')

## 3. READMEに実験実行環境へのリンクを追加する

当実行環境へアクセスするためのリンクを、データガバナンス機能の当実験リポジトリのREADMEに追記します。

In [ ]:
import urllib
# repository_title取得
with tempfile.TemporaryDirectory() as tmpdir:
    # 使い終わったらPython側で消してくれるTemporaryDirectoryを利用
    # refs: https://qiita.com/everylittle/items/aa7c6f612ff0a9db7f01
    tmppath = os.path.join(tmpdir, remote_info)
    !git remote show origin > $tmppath
    with open(tmppath, 'r') as f:
        lines = [s.strip() for s in f.readlines()]
        repository_title = lines[2].split('/')[-1].replace('.git', '')
# remote_http_url生成
remote_http_url = params['siblings']['ginHttp']+name+'/'+repository_title+'.git'

In [ ]:
import os, urllib
from IPython.display import clear_output

%cd ~/
with open('README.md', 'w', newline='\n') as f:
    f.write("\n## 実験実行環境にアクセスしたい場合\n以下のリンクをクリックしてください<br>https://jupyter.cs.rcos.nii.ac.jp" + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + "notebooks/experiment.ipynb  ")
    f.write("\n\n上記リンクからアクセスできない場合は以下のリンクから実験実行環境を再度立ち上げてください  ")
    f.write("\n※「他リポジトリから実験の入力データを用意する」で入力データを用意した場合は、再度必要なファイルの実体をdatalad get　'ファイルパス'で取得する必要があります。  ")
    f.write("\nhttps://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(remote_http_url, safe='') + "/HEAD?filepath=experiment.ipynb")

clear_output()
print('READMEに実行環境へのリンクを追加しました。')

## 4. 高性能実験環境利用のために必要な設定をする

高性能実験環境を利用している場合には、以降のセルを実行してください。

### 4.1 アカウント情報の入力

以下のセルを実行し、表示されるフォームにアカウント情報を入力してください。

In [ ]:
from IPython.display import clear_output
import getpass
name_mdx = input("高性能実験環境におけるSSHユーザ名：")
clear_output()

### 4.2. SSH秘密鍵の設置

左上のバナーを押下し、ファイル一覧画面に遷移してください。  
遷移後、id_rsaファイルをドラッグアンドドロップによりアップロードしてください。  
アップロード後、以下のセルを実行してください。

In [ ]:
!mkdir -p /home/jovyan/.ssh/
!mv ~/id_rsa ~/.ssh/
!chmod 600 ~/.ssh/id_rsa

### 4.3. 高性能実験環境利用のための設定を行う

In [ ]:
%cd ~/EX-WORKFLOW/util/scripts
from scripts import utils
from IPython.display import clear_output

# mdx接続情報を設定ファイルに記述する
utils.config_mdx(name_mdx=name_mdx, mdxDomain=params['mdxDomain'])
clear_output()
print('mdx接続のための準備が出来ました。')

### 4.4. 実験活動のためのディレクトリ初期化

In [ ]:
import os
tmp = os.environ["BINDER_REF_URL"]
tmp = tmp.split("/")
tmp = tmp[len(tmp) - 1]
repo_name = tmp[:tmp.find(".")]

cmd = "mkdir -p ./" + repo_name + "/"
!ssh mdx $cmd

!ssh mdx mkdir -p ./$repo_name/input_data/
!ssh mdx mkdir -p ./$repo_name/output_data/
!ssh mdx mkdir -p ./$repo_name/source/

!scp -r ~/input_data/* mdx:~/$repo_name/input_data/
!scp -r ~/output_data/* mdx:~/$repo_name/output_data/
!scp -r ~/source/* mdx:~/$repo_name/source/
    
print('============================================================')
print('mdx環境への実験データと実験実行コードの転送が完了しました。')

## 5. 実行結果を実験リポジトリに同期する

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
%cd ~/EX-WORKFLOW/util/scripts
from scripts import utils

# SSHホスト（＝GIN）を信頼する設定
utils.config_GIN(ginHttp = params['siblings']['ginHttp'])

In [ ]:
import os
import glob

# Git管理のパスのリストを作成する
%cd ~/
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = files

# Git-annex管理するパスの配列を作成する
annexed_save_path = ['input_data', 'output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = 'ワークフロー実行準備 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False)
)

In [ ]:
import papermill as pm

%cd ~/
# Git管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = 'ワークフロー実行準備 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)

## 6. ワークフロー図を更新する

ワークフロー図にこのワークフローが実行済みであることを反映します。

In [ ]:
%cd ~/
path = EX-WORKFLOW/images/notebooks.diag

with open('.gitignore', 'r') as f:
    text = f.read()
    if text.find(path) == -1:
        !echo "/" + $path >> ./.gitignore

# notebooks.diagのgit管理を外す
!git update-index --skip-worktree $path

In [ ]:
%cd ~/
find = '"required_every_time"[fontsize = 14];'
replace = '"required_every_time"[numbered = 済, fontsize = 14];'

with open(path, 'r') as f:
    s = f.read()

with open(path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 7. 実験中ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../../experiment.ipynb)からトップページに遷移できます。  